# Безпека у Django

## Мета лекції

Після цієї лекції ви:

* зрозумієте основні загрози веб-додатків;
* знатимете вбудовані механізми безпеки Django;
* навчитеся захищати Django-додаток від типових атак;
* знатимете best practices для production;
* розумітимете, на що звертають увагу security та QA engineers.

---

## 1. Чому безпека критично важлива

Веб-додатки постійно піддаються атакам:

* крадіжка даних користувачів
* несанкціонований доступ
* підміна запитів
* витік секретів

📌 **Безпека — це не опція, а обовʼязкова частина розробки**

---

## 2. Основні типи атак

### Найпоширеніші загрози (OWASP Top 10):

* SQL Injection
* Cross-Site Scripting (XSS)
* Cross-Site Request Forgery (CSRF)
* Broken Authentication
* Security Misconfiguration

---

## 3. SQL Injection і Django ORM

### Проблема (raw SQL)

```python
cursor.execute(f"SELECT * FROM users WHERE name = '{username}'")
```

❌ Вразливо

### Рішення: Django ORM

```python
User.objects.get(username=username)
```

✅ Django ORM **автоматично захищає** від SQL Injection

---

## 4. Cross-Site Scripting (XSS)

XSS — це виконання шкідливого JavaScript у браузері користувача.

### Django templates — безпечні за замовчуванням

```html
<p>{{ user_input }}</p>
```

➡ HTML автоматично екранується.

### Небезпечний код

```html
{{ user_input|safe }}
```

❌ Використовуйте `safe` **тільки якщо впевнені у даних**

---

## 5. Cross-Site Request Forgery (CSRF)

CSRF — підміна POST-запиту від імені користувача.

### Захист у Django

```html
<form method="post">
    {% csrf_token %}
</form>
```

❌ Без CSRF → 403 Forbidden

📌 Django включає CSRF-захист **за замовчуванням**

---

## 6. Аутентифікація та паролі

### Хешування паролів

```python
user.set_password('secret')
user.save()
```

❌ Ніколи не зберігайте паролі у відкритому вигляді

### Перевірка пароля

```python
user.check_password('secret')
```

---

## 7. Permissions та доступ

### Декоратор login_required

```python
from django.contrib.auth.decorators import login_required

@login_required
def profile(request):
    ...
```

### Permissions

```python
from django.contrib.auth.decorators import permission_required

@permission_required('blog.add_post')
def add_post(request):
    ...
```

---

## 8. Захист адміністративної панелі

### Зміна URL admin

```python
path('secure-admin/', admin.site.urls)
```

### Обмеження доступу

* сильні паролі
* 2FA (опціонально)
* обмеження IP (через Nginx)

---

## 9. Security settings для production

```python
DEBUG = False
SECURE_SSL_REDIRECT = True
SESSION_COOKIE_SECURE = True
CSRF_COOKIE_SECURE = True
SECURE_BROWSER_XSS_FILTER = True
SECURE_CONTENT_TYPE_NOSNIFF = True
X_FRAME_OPTIONS = 'DENY'
```

📌 HTTPS — обовʼязковий

---

## 10. SECRET_KEY та environment variables

❌ Погано:

```python
SECRET_KEY = 'hardcoded'
```

✅ Добре:

```python
import os
SECRET_KEY = os.environ.get('SECRET_KEY')
```

---

## 11. Захист API (коротко, DRF)

```python
from rest_framework.permissions import IsAuthenticated

permission_classes = [IsAuthenticated]
```

### Token auth (приклад)

```python
REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': [
        'rest_framework.authentication.TokenAuthentication',
    ]
}
```

---

## 12. Логування та аудит

```python
LOGGING = {
    'version': 1,
    'handlers': {
        'file': {
            'class': 'logging.FileHandler',
            'filename': 'security.log',
        },
    },
}
```

📌 Логи допомагають знаходити атаки

---

## 13. Оновлення залежностей

```bash
pip list --outdated
pip install -U django
```

📌 Старі версії = вразливості

---

## 14. Типові помилки

❌ DEBUG = True у production

❌ SECRET_KEY у репозиторії

❌ Відсутній CSRF

❌ Використання `safe`

---

## 15. Коротке резюме

* Django безпечний за замовчуванням
* ORM → захист від SQL Injection
* Templates → захист від XSS
* CSRF → must-have
* HTTPS + security settings

---

## Практичне завдання

1. Увімкніть security settings
2. Додайте login_required до view
3. Захистіть форму CSRF
4. Перевірте, що DEBUG = False
